## 3. Дисбаланс выборки

In [4]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import plotly
import plotly.express as px

from sklearn import metrics 
from sklearn import model_selection
from sklearn import linear_model
from sklearn import tree


In [5]:
water_data = pd.read_csv("data/water_potability.csv")
water_data.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0


In [7]:
display(water_data["Potability"].value_counts(normalize=True))

0    0.60989
1    0.39011
Name: Potability, dtype: float64

In [10]:
fig1 = px.histogram(
    data_frame=water_data["Potability"]
)
fig1.show()